In [ ]:
# This notebook is an attempt to train a neural net to recognize my handwriting
# Each instance will be trained for a specific letter
import numpy

# Represents a class for a specific recognition function.
class Neuron:
  def __init__(self):
    pass
    
    def _nonlin(x, deriv=False):        
        if(deriv==True):
            return x*(1-x)

    return 1/(1+np.exp(-x))
    
    def train(self, data, iterations = 10000):
    

